# Notebook to check direct db calls against Bovas's NAV report code

In [34]:
%reload_ext autoreload
%autoreload 
import sys, os
project_root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root_path not in sys.path:
    sys.path.append(project_root_path)
print(project_root_path)

import datetime as dt
import numpy as np
import pandas as pd
import plotly.express as px

import pymd 
import nav.utils
from qpt_historic_pos.impl.utils.times import ChicagoTimeZone, UtcTimeZone
import qpt_stress_test.apps.generate_daily_summary as generate_daily_summary
import qpt_stress_test.services.positions as positions
import qpt_stress_test.core.config as config
import qpt_stress_test.core.qpt_config as qpt_config
import qpt_stress_test.db.tasks as db_tasks
import qpt_stress_test.db.repositories.drivers.pyodbc as pyodbc
import qpt_stress_test.db.repositories.drivers.sqlalchemy as sqlalchemy
import qpt_stress_test.db.repositories.qpt_pg as qpt_pg
import qpt_stress_test.db.repositories.qpt_mssql as qpt_mssql
import qpt_stress_test.db.repositories.databricks as db_trading
from qpt_stress_test.db.tasks import bfc_rds_sqlalchemy_engine_factory, sv_awoh_dw01_pyodbc_connection_factory,  gdt_cluster_databricks_connection_factory

from importlib import reload
reload(positions)
reload(config)
reload(qpt_config)
reload(db_tasks)

pymd.enable_logging()

c:\Users\skingham\Projects\galaxysk\qpt_stress_test


In [35]:
# Set dates, etc, for report generation
nav_date = dt.date(2023, 1, 4)
nav_00utc = dt.datetime.combine(nav_date + dt.timedelta(days=1), dt.time(hour=0, minute=0, second=0))
nav_ctz = UtcTimeZone.localize(dt.datetime.combine(nav_date + dt.timedelta(days=1), dt.time(hour=0, minute=0, second=0))).astimezone(ChicagoTimeZone)
derivs_utc_datetime = ChicagoTimeZone.localize(dt.datetime.combine(nav_date, dt.time(hour=16, minute=0, second=0))).astimezone(UtcTimeZone)

## Get Summary Exchange Balances used in Bovas's NAV report

In [36]:
%%capture
# This code is copied from the generate_daily_summary.py app 
mktdata_repo = qpt_mssql.MarketDataRepository(sql_query_driver=pyodbc.SqlQuery, db_connector_factory=sv_awoh_dw01_pyodbc_connection_factory)
operations_repo = qpt_mssql.OperationsRepository(sql_query_driver=pyodbc.SqlQuery, db_connector_factory=sv_awoh_dw01_pyodbc_connection_factory)
trading_repo = qpt_mssql.TradingRepository(sql_query_driver=pyodbc.SqlQuery, db_connector_factory=sv_awoh_dw01_pyodbc_connection_factory)
exchange_repo = qpt_mssql.RawDataRepository(sql_query_driver=pyodbc.SqlQuery, db_connector_factory=sv_awoh_dw01_pyodbc_connection_factory)

# Run Bowen's Net open positions report: should match email's from Bcai and Wenbo's emails:
net_open_position_report_df = generate_daily_summary.net_open_positions(derivs_utc_datetime, trading_repo)                                                             

# Run a facsimile of the Bovas NAV report: should  match Bovas NAV Summary_Exchange_Balances_00UTC worksheet
assets_df, loans_df, summary = generate_daily_summary.generate_daily_nav_00utc(nav_date)
summary_exchange_balances_df = generate_daily_summary.summary_exchange_balances_00utc(nav_date, assets_df, loans_df, summary)

## Reformat into format to combine nav & deriv positions
asset_loans_cash_df = generate_daily_summary.summary_asset_loans_cash(summary_exchange_balances_df)
asset_and_open_positions_df = pd.concat([net_open_position_report_df, asset_loans_cash_df], ignore_index=True)

In [37]:
# Take Bovas NAV report, un-mangle accounts
# ['Account','Balance','BalanceType','Currency','Source','Timestamp','Timestamp_Native','Notional','REFERENCE 1','REFERENCE 2','TYPE','Endpoint']
reported_eod_balances = summary_exchange_balances_df.copy()
reported_eod_balances.loc[(reported_eod_balances['Balance'] < 0) & (reported_eod_balances['BalanceType'] == ''), 'BalanceType'] = 'Margin Loan'
reported_eod_balances.loc[reported_eod_balances['Balance'] < 0, 'Account'] = reported_eod_balances.loc[reported_eod_balances['Balance'] < 0, 'Account'].replace(
    ['HUBI-M Margin Loan','HUB2-M Margin Loan','WOOX-1-M-E Margin Loan','FTXE-1-M-E Margin Loan', 'OKEX-2-M Margin Loan','OKEX-2-S3 Margin Loan','BINE-2-S1-M Margin Loan'],
    ['HUBI-M','HUB2-M','WOOX-1-M-E','FTXE-1-M-E', 'OKEX-2-M','OKEX-2-S3','BINE-2-S1-M'])

print(f"Bovas Net Asset Value w/o GD Capital: {summary_exchange_balances_df.Notional.sum():,.0f}; "
      f"Total NAV & Net Open Positions balance: {asset_and_open_positions_df.notional.sum():,.0f}")

usd_currencies = ('USD', 'USDT', 'USDC')
usd_like_balances = reported_eod_balances.loc[reported_eod_balances['Currency'].isin(usd_currencies)]
balances_ex_usd = reported_eod_balances.loc[~reported_eod_balances['Currency'].isin(usd_currencies)]
print(f"  {usd_currencies} balance: {usd_like_balances.Notional.sum():,.0f};  NAV excl USD like balances: {balances_ex_usd.Notional.sum():,.0f}")

#['utc_timestamp', 'exchange', 'account', 'instrument', 'position', 'notional', 'mark_price', 'unrealized_pnl', 'instrument_type', 'expiration_time', 'is_linear', 'underlying']
nav_by_account = asset_and_open_positions_df[['exchange', 'notional']].groupby(by=['exchange']).sum(numeric_only=True).reset_index()
display(nav_by_account)

nav_by_curency = reported_eod_balances[['Timestamp', 'Currency', 'Balance', 'Notional']].groupby(by=['Timestamp', 'Currency',]).sum(numeric_only=True).reset_index()
display(nav_by_curency)

# [Timestamp], [Account], [Currency], [Source], [BalanceType]	Notional Balance
nav_by_account_ccy = reported_eod_balances[['Timestamp', 'Endpoint', 'Currency', 'Source', 'BalanceType', 'Balance', 'Notional']].groupby(by=['Timestamp', 'Endpoint', 'Currency', 'Source', 'BalanceType']).sum(numeric_only=True).reset_index()
display(nav_by_account_ccy)

Bovas Net Asset Value w/o GD Capital: 32,047,940; Total NAV & Net Open Positions balance: 11,912,607
  ('USD', 'USDT', 'USDC') balance: 21,297,699;  NAV excl USD like balances: 10,750,241


,exchange,notional
0,BINANCE,"14,559,017.85"
1,BTFX,160.12
2,BTSE,"157,674.21"
3,BULL,"48,367,717.43"
4,CASH,"699,614.89"
5,DEFI,"4,541,041.79"
6,DYDX,"-8,726.52"
7,ED&F,"76,598,881.22"
8,FBLK,"37,237,163.74"
9,GALAXY,"-189,130,431.40"


,Timestamp,Currency,Balance,Notional
0,20230104,$ PLEASE VISIT ACADY.SITE TO RECEIVE REWARD.,"180,000.00",0.00
1,20230104,$ USDCBONUS.COM <- VISIT TO CLAIM BONUS,"7,000.00",0.00
2,20230104,$ USDCGIFT.COM <- VISIT TO CLAIM BONUS,"7,000.00",0.00
3,20230104,$ USDCXMAS.COM <- VISIT TO CLAIM GIFT,"7,000.00",0.00
4,20230104,927$ VISIT AUSDBONUS.COM TO CLAIM,8.00,0.00
5,20230104,AAC,377.10,0.10
6,20230104,AAVE,192.00,"10,903.36"
7,20230104,ADA,"-16,322.80","-4,369.25"
8,20230104,ALGO,"3,628.71",678.90
9,20230104,ARPA,0.53,0.01


,Timestamp,Endpoint,Currency,Source,BalanceType,Balance,Notional
0,20230104,BINE,AAVE,bine_2_s1_e.account_asset,,569.39,"32,334.51"
1,20230104,BINE,AAVE,bine_2_s1_m.margin_account_asset,Margin Loan,-900.00,"-51,108.75"
2,20230104,BINE,ADA,bine_2_s1_e.account_asset,,"280,687.95","75,133.84"
3,20230104,BINE,ADA,bine_2_s1_m.margin_account_asset,Margin Loan,"-450,000.00","-120,454.85"
4,20230104,BINE,ALGO,bine_2_s1_e.account_asset,,"213,161.09","39,880.85"
5,20230104,BINE,ALGO,bine_2_s1_m.margin_account_asset,Margin Loan,"-244,800.00","-45,800.26"
6,20230104,BINE,ATOM,bine_2_s1_e.account_asset,,0.00,0.04
7,20230104,BINE,AVAX,bine_2_s1_e.account_asset,,"3,220.03","38,890.26"
8,20230104,BINE,AVAX,bine_2_s1_m.margin_account_asset,Margin Loan,"-6,000.00","-72,465.68"
9,20230104,BINE,BAT,bine_2_s1_e.account_asset,,"78,517.63","14,170.15"


## Reconcile Bovas totals

Let's just take the queries that Bovas uses to get crypto exchange asset & loan entries and look at all entries for 'Operations.balances.EndOfDay_00UTC'.  What's missing?

In [38]:
# Bovas config
bovas_asset_account_name_replacement = (
    ['OKEX-2-W1', 'OKEX-2-U1', 'FBLK-AUDIT-BINE', 'FBLK-AUDIT-HUBI', 'FBLK-AUDIT-OKEX', 'FBLK-Default', 'FBLK-GOTC-GACM', 'FBLK-LEND-BTGO', 'FBLK-LEND-DRAW', 
     'FBLK-LEND-OXTF', 'FBLK-LEND-XRPF', 'FBLK-LMAC-M', 'FBLK-Network Deposits', 'FBLK-PITX-E', 'FBLK-LEND-CELS', 'FBLK-LEND-GADI', 'FBLK-LEND-NICO', 
     'FBLK-LEND-GENX','FBLK-BINE-MX-S1','FBLK-DEFI-AAVE','FBLK-GOTC-BIGO','FBLK-DYDX-1-M-P','FBLK-WOOX-1-M-E','FBLK-LEND-GACM'], 
    ['OKEX-2-M-W', 'OKEX-2-M', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK','FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 'FBLK', 
     'FBLK', 'FBLK', 'FBLK', 'FBLK'])

bovas_loan_account_name_replacement = (
    ['HUBI-M','HUB2-M','WOOX-1-M-E','FTXE-1-M-E','OKEX-2-U1','OKEX-2-S3','BINE-2-S1-M'],
    ['HUBI-M Margin Loan','HUB2-M Margin Loan','WOOX-1-M-E Margin Loan','FTXE-1-M-E Margin Loan', 
     'OKEX-2-M Margin Loan','OKEX-2-S3 Margin Loan','BINE-2-S1-M Margin Loan'])

bovas_currency_replacement = (
    ['USDT_ERC20','SRM_LOCKED','BTCUSD','ETHUSD','EOSUSD','LINKUSD','LTCUSD','ATOM.S','DOT.S','KSM.S','FTM_FANTOM','AUSDC_ETH',
     'CVXCRV-F','VARIABLEDEBTCRV','BNB_BSC','ZIL_BSC','EUROC_ETH_F5NG','USDTEST','FRXETH','AAVAUSDC','VARIABLEDEBTAVAUSDT',
     'VARIABLEDEBTUSDT','AURAB-STETH-STABLE-VAULT'],
    ['USDT','SRM','BTC','ETH','EOS','LINK','LTC','ATOM','DOT','KSM','FTM','AUSDC','CVX','CRV','BNB','ZIL','EUROC','USD','ETH',
    'AUSDC','USDT','USDT','ETH'])

In [39]:
%%capture
# Do a direct call against the database:
#
marks_df = mktdata_repo.get_coinmarketcap_close_marks(nav_date).as_dataframe()
# Add in EUR, GBP, etc missing from coinbase
marks2_df = mktdata_repo.get_trading_close_marks(nav_date).as_dataframe()
marks_df = pd.concat([marks_df, marks2_df.loc[~marks2_df['Currency'].isin(marks_df.Currency)]], ignore_index=True)
# Drop duplicates
marks_df.Currency = marks_df.Currency.apply(str.upper)
marks_df.drop(marks_df[marks_df.Currency.duplicated(keep="first")].index, axis=0, inplace=True)
# adding coins which are having a different name but essentially same as an existing coin
marks_df.set_index('Currency', inplace=True)
marks_df.loc['AUSDC'] = marks_df.loc['USDC'].values
marks_df.loc['AWETH'] = marks_df.loc['WETH'].values
marks_df.loc['FRXETH'] = marks_df.loc['ETH'].values

In [40]:
%%capture
bovas_loans_qry = """
select a.Account, a.Balance, a.BalanceType, UPPER(a.Currency) as Currency, 
    Isnull(b.TableName, '') as Source, FORMAT(a.Date_UTC,'yyyyMMdd') as Timestamp, a.AsOf_UTC as Timestamp_Native
from Operations.balances.EndOfDay_00UTC A 
    LEFT JOIN Operations.balances.sources B on a.Account = b.Account 
where balance != 0 
    and a.Date_UTC = '{date:%Y-%m-%d}' 
    and Currency not like '%SWAP%' 
    and a.Account in {accounts} 
    and balancetype != 'unrealized' 
order by Account
""".format(date=nav_date, accounts=tuple(qpt_config.loan_accounts))
bovas_loans_df = trading_repo.adhoc_query(bovas_loans_qry).as_dataframe()

bovas_assets_qry = """
select a.Account, a.Balance, a.BalanceType, UPPER(a.Currency) as Currency,  
    Isnull(b.TableName, '') as Source, FORMAT(a.Date_UTC,'yyyyMMdd') as Timestamp, a.AsOf_UTC as Timestamp_Native 
from Operations.balances.EndOfDay_00UTC A 
    LEFT JOIN Operations.balances.sources B on a.Account = b.Account 
where balance != 0 and a.Date_UTC = '{date:%Y-%m-%d}' 
    and Currency not like '%SWAP%' 
    and a.Account in {accounts} 
    and Seconds_from_00UTC <= 79259
order by Account
""".format(date=nav_date, accounts=tuple(qpt_config.exchange_balance_accounts))
bovas_assets_df = trading_repo.adhoc_query(bovas_assets_qry).as_dataframe()

DatabaseError: Execution failed on sql '
select a.Account, a.Balance, a.BalanceType, UPPER(a.Currency) as Currency, 
    Isnull(b.TableName, '') as Source, FORMAT(a.Date_UTC,'yyyyMMdd') as Timestamp, a.AsOf_UTC as Timestamp_Native
from Operations.balances.EndOfDay_00UTC A 
    LEFT JOIN Operations.balances.sources B on a.Account = b.Account 
where balance != 0 
    and a.Date_UTC = '2023-01-04' 
    and Currency not like '%SWAP%' 
    and a.Account in ('LEND-GACM', 'LEND-HUBI', 'LEND-LMAC', 'LEND-PITX', 'LEND-XRPF', 'LEND-OKEX', 'HUBI-3-S3-M Loan', 'HUBI-M', 'HUB2-M', 'LEND-OXTF', 'WOOX-1-M-E', 'FTXE-1-M-E', 'OKEX-2-U1', 'LEND-HUBS6', 'BULL-1-M-M Loan', 'BULL-2-M-M Loan', 'LEND-BULL', 'OKEX-2-S3', 'BINE-2-S1-M', 'DEFI-STRAT-1 Loan', 'GATE-1-M-M Loan', 'BINE-2-S2-M', 'DEFI-STRAT-8 Loan') 
    and balancetype != 'unrealized' 
order by Account
': ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionWrite (send()). (10054) (SQLExecDirectW); [01000] [Microsoft][ODBC SQL

In [ ]:
%%capture
# Operations EOD 
op_eod_balances_df = operations_repo.get_eod_balances(nav_date).as_dataframe()
op_eod_balances_df.Currency.replace(*bovas_currency_replacement, inplace=True)
op_eod_balances_df['Notional'] = op_eod_balances_df.Balance * marks_df.reindex(op_eod_balances_df.Currency).fillna(0).Mark.values

In [ ]:
%%capture
# Exchange balances
#cme_exchange_df = exchange_repo.get_cme_positions(at_dtt_utc=nav_00utc).as_dataframe()
crypto_exchange_df = exchange_repo.get_exchange_balances(asof=nav_ctz).as_dataframe()
crypto_exchange_df['Notional'] = crypto_exchange_df.Balance * marks_df.reindex(crypto_exchange_df.Currency).fillna(0).Mark.values


In [ ]:
%%capture
# We should properly map accounts to Endpoint or exchange
bovas_assets_df['Endpoint'] = bovas_assets_df['Account'].apply(lambda ac: str(ac.split('-')[0]) if str(ac.split('-')[0]) != 'LEND' else str(ac.split('-')[1]))
bovas_loans_df['Endpoint'] = bovas_loans_df['Account'].apply(lambda ac: str(ac.split('-')[0]) if str(ac.split('-')[0]) != 'LEND' else str(ac.split('-')[1]))
op_eod_balances_df['Endpoint'] = op_eod_balances_df['Account'].apply(lambda ac: str(ac.split('-')[0]) if str(ac.split('-')[0]) != 'LEND' else str(ac.split('-')[1]))
crypto_exchange_df['Endpoint'] = crypto_exchange_df['Account'].apply(lambda ac: str(ac.split('-')[0]) if str(ac.split('-')[0]) != 'LEND' else str(ac.split('-')[1]))

# Add in bank balances; 
bank_eod_balances_df = trading_repo.get_bank_balances(nav_date).as_dataframe()
bank_eod_balances_df['Notional'] = bank_eod_balances_df.Balance * marks_df.reindex(bank_eod_balances_df.Currency).fillna(0).Mark.values
bank_eod_balances_df['Endpoint'] = 'CASH'
bovas_assets_df = pd.concat([bovas_assets_df, bank_eod_balances_df], ignore_index=True)
op_eod_balances_df = pd.concat([op_eod_balances_df, bank_eod_balances_df], ignore_index=True)


In [ ]:
%%capture
# Add in ED&F collateral manually
edf_cash = generate_daily_summary.get_edf_cash_with_fallback(nav_date)
edf_cash_df = pd.DataFrame.from_dict({
        'Account': ['ED&F Man Capital'], 
        'Balance': [edf_cash], 
        'BalanceType': ['Balance'], 
        'Currency': ['USD'], 
        'Source': [''],
        'Timestamp': [f"{nav_date:%Y%m%d}"],
        'Timestamp_Native': [nav_00utc],
        'Notional': [edf_cash], 
        'Endpoint': ['ED&F']
 })

op_eod_balances_df.drop(op_eod_balances_df.loc[op_eod_balances_df.Account == 'ED&F Man Capital'].index, axis=0, inplace=True)
op_eod_balances_df = pd.concat([op_eod_balances_df, edf_cash_df], ignore_index=True)

bovas_assets_df.drop(bovas_assets_df.loc[bovas_assets_df.Account == 'ED&F Man Capital'].index, axis=0, inplace=True)
bovas_assets_df = pd.concat([bovas_assets_df, edf_cash_df], ignore_index=True)

edf_cash_df.loc[0, 'Timestamp'] = f"{nav_date:%Y-%m-%d}"
crypto_exchange_df.drop(crypto_exchange_df.loc[crypto_exchange_df.Account == 'ED&F Man Capital'].index, axis=0, inplace=True)
crypto_exchange_df = pd.concat([crypto_exchange_df, edf_cash_df], ignore_index=True)

In [ ]:
# display(marks_df)
# display(bovas_loans_df)
# display(bovas_assets_df)
# display(op_eod_balances_df)

# Display DF of all records not used in Bovas's report
op_eod = op_eod_balances_df.copy()
bovas_assets_eod = bovas_assets_df.copy()
bovas_assets_eod = nav.utils._format_assets(nav_date, bovas_assets_eod, marks_df)
bovas_loans_eod = bovas_loans_df.copy()
bovas_loans_eod = nav.utils._format_loans(nav_date, bovas_loans_eod, marks_df)

# Assets - apply the mappings to account names that Bovas's code does
op_eod.loc[((op_eod['Balance'] > 0) | (op_eod['BalanceType'] == 'Unrealized')), 'Account'] \
      = op_eod.loc[((op_eod['Balance'] > 0) | (op_eod['BalanceType'] == 'Unrealized')), 'Account'].replace(*bovas_asset_account_name_replacement)
#bovas_assets_eod.Account.replace(*bovas_asset_account_name_replacement, inplace=True)
#bovas_assets_eod.Currency.replace(*bovas_currency_replacement, inplace=True)

# Loans
op_eod.loc[((op_eod['Balance'] < 0) & (op_eod['BalanceType'] != 'Unrealized')), 'Account'] \
      = op_eod.loc[((op_eod['Balance'] < 0) & (op_eod['BalanceType'] != 'Unrealized')), 'Account'].replace(*bovas_loan_account_name_replacement)
#bovas_loans_eod.Account.replace(*bovas_loan_account_name_replacement, inplace=True)
#bovas_loans_eod.Currency.replace(*bovas_currency_replacement, inplace=True)

# Sum Fireblocks to currency
op_eod = nav.utils._format_fireblocks(op_eod)

df1 = pd.concat([op_eod, bovas_assets_eod, bovas_loans_eod]).set_index(op_eod.columns.tolist())
df2 = pd.concat([bovas_assets_eod, bovas_loans_eod]).set_index(op_eod.columns.tolist())
bovas_ignored_df = df1.loc[df2.index.symmetric_difference(df1.index)].reset_index()

# Get all the bovas cypto position records, without the account name mangling, into one df
#op_eod = op_eod_balances_df.copy()
df1 = pd.concat([op_eod, bovas_ignored_df]).set_index(op_eod.columns.tolist())
df2 = bovas_ignored_df.set_index(op_eod.columns.tolist())
bovas_included_df = df1.loc[df2.index.symmetric_difference(df1.index)].reset_index()

# Three views of the exchange positions
reported_eod_df = reported_eod_balances.copy()
reported_eod_df['Unrealized'] = reported_eod_df.apply(lambda row: row.Notional if row.BalanceType == 'Unrealized' else 0, axis=1)
reported_eod_df['Loans-funding'] = reported_eod_df.apply(lambda row: row.Notional if row.BalanceType != 'Unrealized' and row.Notional < 0 else 0, axis=1)
reported_eod_df['Assets'] = reported_eod_df.apply(lambda row: row.Notional if  row.BalanceType != 'Unrealized' and row.Notional > 0 else 0, axis=1)
report_totals = reported_eod_df[['Timestamp', 'Endpoint', 'Loans-funding', 'Assets', 'Unrealized', 'Notional']].groupby(by=['Timestamp',]).sum(numeric_only=True).reset_index()
report_by_exchange = reported_eod_df[['Timestamp', 'Endpoint', 'Loans-funding', 'Assets', 'Unrealized', 'Notional']].groupby(by=['Timestamp', 'Endpoint',]).sum(numeric_only=True).reset_index()

bovas_df = pd.concat([bovas_assets_eod, bovas_loans_eod], ignore_index=True)
#bovas_df['Endpoint'] = bovas_df['Account'].apply(lambda ac: str(ac.split('-')[0]) if str(ac.split('-')[0]) != 'LEND' else str(ac.split('-')[1]))
bovas_df['Unrealized'] = bovas_df.apply(lambda row: row.Notional if row.BalanceType == 'Unrealized' else 0, axis=1)
bovas_df['Loans-funding'] = bovas_df.apply(lambda row: row.Notional if row.BalanceType != 'Unrealized' and row.Notional < 0 else 0, axis=1)
bovas_df['Assets'] = bovas_df.apply(lambda row: row.Notional if  row.BalanceType != 'Unrealized' and row.Notional > 0 else 0, axis=1)
bovas_totals = bovas_df[['Timestamp', 'Endpoint', 'Loans-funding', 'Assets', 'Unrealized', 'Notional']].groupby(by=['Timestamp',]).sum(numeric_only=True).reset_index()
bovas_by_exchange = bovas_df[['Timestamp', 'Endpoint', 'Loans-funding', 'Assets', 'Unrealized', 'Notional']].groupby(by=['Timestamp', 'Endpoint',]).sum(numeric_only=True).reset_index()

op_eod_df = op_eod.copy()
#op_eod_df['Endpoint'] = op_eod_df['Account'].apply(lambda ac: str(ac.split('-')[0]) if str(ac.split('-')[0]) != 'LEND' else str(ac.split('-')[1]))
op_eod_df['Unrealized'] = op_eod_df.apply(lambda row: row.Notional if row.BalanceType == 'Unrealized' else 0, axis=1)
op_eod_df['Loans-funding'] = op_eod_df.apply(lambda row: row.Notional if row.BalanceType != 'Unrealized' and row.Notional < 0 else 0, axis=1)
op_eod_df['Assets'] = op_eod_df.apply(lambda row: row.Notional if  row.BalanceType != 'Unrealized' and row.Notional > 0 else 0, axis=1)
op_eod_totals = op_eod_df[['Timestamp', 'Endpoint', 'Loans-funding', 'Assets', 'Unrealized', 'Notional']].groupby(by=['Timestamp',]).sum(numeric_only=True).reset_index()
op_eod_by_exchange = op_eod_df[['Timestamp', 'Endpoint', 'Loans-funding', 'Assets', 'Unrealized', 'Notional']].groupby(by=['Timestamp', 'Endpoint',]).sum(numeric_only=True).reset_index()

crypto_eod_df = crypto_exchange_df.copy()
#op_eod_df['Endpoint'] = op_eod_df['Account'].apply(lambda ac: str(ac.split('-')[0]) if str(ac.split('-')[0]) != 'LEND' else str(ac.split('-')[1]))
crypto_eod_df['Unrealized'] = crypto_eod_df.apply(lambda row: row.Notional if row.BalanceType == 'Unrealized' else 0, axis=1)
crypto_eod_df['Loans-funding'] = crypto_eod_df.apply(lambda row: row.Notional if row.BalanceType != 'Unrealized' and row.Notional < 0 else 0, axis=1)
crypto_eod_df['Assets'] = crypto_eod_df.apply(lambda row: row.Notional if  row.BalanceType != 'Unrealized' and row.Notional > 0 else 0, axis=1)
crypto_eod_totals = crypto_eod_df[['Timestamp', 'Endpoint', 'Loans-funding', 'Assets', 'Unrealized', 'Notional']].groupby(by=['Timestamp',]).sum(numeric_only=True).reset_index()
crypto_eod_by_exchange = crypto_eod_df[['Timestamp', 'Endpoint', 'Loans-funding', 'Assets', 'Unrealized', 'Notional']].groupby(by=['Timestamp', 'Endpoint',]).sum(numeric_only=True).reset_index()


print(f"Total Exchange EOD notional balance reported in Bovas's report:  {bovas_included_df.Notional.sum(numeric_only=True):,.0f}"
      f"\tNotional notional balance of EOD records removed from reporting: {bovas_ignored_df.Notional.sum(numeric_only=True):,.0f}\n")

print("Summary from Bovas's code")
display(report_totals)
display(report_by_exchange)

print("Summary from bovas's sql")
display(bovas_totals)
display(bovas_by_exchange)

print("Summary raw data exchange balance tables.")
display(crypto_eod_totals)
display(crypto_eod_by_exchange)

print("Summary from ops table - this shows the size of the effect of double counting, etc, that happens if we ignore finance's exclusions.")
display(op_eod_totals)
display(op_eod_by_exchange)

Total Exchange EOD notional balance reported in Bovas's report:  35,170,344	Notional notional balance of EOD records removed from reporting: 58,989,496

Summary from Bovas's code


c:\Users\skingham\Projects\galaxysk\qpt_stress_test\nav\utils.py:99: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  temp = temp.groupby(['Account','Currency','BalanceType','Source','Timestamp','Timestamp_Native']).sum().reset_index()
c:\Users\skingham\Projects\galaxysk\qpt_stress_test\nav\utils.py:100: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp, ignore_index=True)
c:\Users\skingham\Projects\galaxysk\qpt_stress_test\nav\utils.py:99: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  temp = temp.groupby(['A

,Timestamp,Loans-funding,Assets,Unrealized,Notional
0,20230110,"-301,615,194.31","338,102,595.78","-1,097,138.03","35,390,263.43"


,Timestamp,Endpoint,Loans-funding,Assets,Unrealized,Notional
0,20230110,BINE,"-1,766,315.47","76,697,936.93","-900,173.93","74,031,447.53"
1,20230110,BTFX,0.00,166.39,0.00,166.39
2,20230110,BTSE,0.00,"158,913.08",0.00,"158,913.08"
3,20230110,BULL,"-71,253,057.90","119,708,558.04",0.00,"48,455,500.14"
4,20230110,CASH,0.00,"726,247.51",0.00,"726,247.51"
5,20230110,DEFI,"-9,992,297.29","14,655,162.85",0.00,"4,662,865.56"
6,20230110,DYDX,0.00,"18,086.75",0.00,"18,086.75"
7,20230110,ED&F,0.00,"32,067,740.46",0.00,"32,067,740.46"
8,20230110,FBLK,0.00,"38,497,026.30",0.00,"38,497,026.30"
9,20230110,GALAXY,"-207,016,912.36","13,125,191.51",0.00,"-193,891,720.85"


Summary from bovas's sql


...

,Timestamp,Loans-funding,Assets,Unrealized,Notional
0,2023-01-10,"-1,892,248.68","296,287,232.02","-1,067,542.67","293,327,440.67"
1,2023-01-12,0.00,"16,969.02",0.00,"16,969.02"


,Timestamp,Endpoint,Loans-funding,Assets,Unrealized,Notional
0,2023-01-10,BINE,"-1,768,213.55","76,699,830.25","-902,909.35","74,028,707.35"
1,2023-01-10,BTFX,0.00,201.28,0.00,201.28
2,2023-01-10,BTSE,0.00,"158,974.81",0.00,"158,974.81"
3,2023-01-10,BULL,0.00,"119,708,498.00",0.00,"119,708,498.00"
4,2023-01-10,ED&F,0.00,"32,067,740.46",0.00,"32,067,740.46"
5,2023-01-10,FBLK,0.00,"25,216,441.83",0.00,"25,216,441.83"
6,2023-01-10,FUB2,0.00,21.21,0.00,21.21
7,2023-01-10,FUBI,0.00,683.40,0.00,683.40
8,2023-01-10,HUB2,0.00,3.26,0.00,3.26
9,2023-01-10,HUBI,0.00,"12,153,132.37",-324.30,"12,152,808.08"


Summary from ops table - this shows the size of the effect of double counting, etc, that happens if we ignore finance's exclusions.


,Timestamp,Loans-funding,Assets,Unrealized,Notional
0,20230110,"-317,171,330.62","412,428,308.50","-1,097,138.03","94,159,839.85"


,Timestamp,Endpoint,Loans-funding,Assets,Unrealized,Notional
0,20230110,BINE,"-1,766,493.18","76,698,041.33","-900,173.93","74,031,374.22"
1,20230110,BTFX,0.00,166.39,0.00,166.39
2,20230110,BTGO,"-3,489,294.15",0.00,0.00,"-3,489,294.15"
3,20230110,BTSE,0.00,"158,913.08",0.00,"158,913.08"
4,20230110,BULL,"-71,253,057.90","119,708,558.04",0.00,"48,455,500.14"
5,20230110,CASH,0.00,"729,344.04",0.00,"729,344.04"
6,20230110,DEFI,"-9,992,297.29","37,818,558.71",0.00,"27,826,261.42"
7,20230110,DYDX,0.00,"18,086.75",0.00,"18,086.75"
8,20230110,ED&F,0.00,"32,067,740.46",0.00,"32,067,740.46"
9,20230110,FBLK,0.00,"22,297,524.33",0.00,"22,297,524.33"


In [ ]:
print("Operations.balances.EndOfDay_00UTC records that are not included in Bovas's NAV report:")
display(bovas_ignored_df.sort_values(by=['Account', 'Currency'], ignore_index=True))

print("\nOperations.balances.EndOfDay_00UTC records that are included in Bovas's NAV report:")
display(bovas_included_df.sort_values(by=['Account', 'Currency'], ignore_index=True))

Operations.balances.EndOfDay_00UTC records that are not included in Bovas's NAV report:


,Account,Balance,BalanceType,Currency,Source,Timestamp,Timestamp_Native,Notional,Endpoint
0,DEFI-STRAT-4,628.99,reward_token,AURA,,20230110,2023-01-11 00:00:00.497,964.43,DEFI
1,DEFI-STRAT-4,175.24,reward_token,BAL,,20230110,2023-01-11 00:00:00.497,994.73,DEFI
2,DEFI-STRAT-4,414.32,supply_token,ETH,,20230110,2023-01-11 00:00:00.497,"553,780.42",DEFI
3,DEFI-STRAT-4,598.84,supply_token,STETH,,20230110,2023-01-11 00:00:00.497,"787,856.10",DEFI
4,DEFI-STRAT-8,"12,008,983.85",wallet,AUSDC,,20230110,2023-01-11 00:00:00.497,"12,008,833.35",DEFI
5,DEFI-STRAT-8,1.88,wallet,AVAX,,20230110,2023-01-11 00:00:00.497,23.89,DEFI
6,DEFI-STRAT-8,0.46,wallet,ETH,,20230110,2023-01-11 00:00:00.497,619.54,DEFI
7,DEFI-STRAT-8,"33,055.56",wallet,USDT,,20230110,2023-01-09 16:00:00.287,"33,058.86",DEFI
8,DEFI-STRAT-8,"9,991,297.76",wallet,USDT,,20230110,2023-01-11 00:00:00.497,"9,992,297.29",DEFI
9,FBLK-DEFI-STRAT001,9.78,,CRV,,20230110,2023-01-10 23:56:26.857,6.36,FBLK



Operations.balances.EndOfDay_00UTC records that are included in Bovas's NAV report:


,Account,Balance,BalanceType,Currency,Source,Timestamp,Timestamp_Native,Notional,Endpoint
0,BINE-2-S1-E,597.65,,AAVE,bine_2_s1_e.account_asset,20230110,2023-01-11 00:00:00.097,"36,960.37",BINE
1,BINE-2-S1-E,"305,794.89",,ADA,bine_2_s1_e.account_asset,20230110,2023-01-11 00:00:00.097,"98,617.54",BINE
2,BINE-2-S1-E,"206,808.09",,ALGO,bine_2_s1_e.account_asset,20230110,2023-01-11 00:00:00.097,"42,203.53",BINE
3,BINE-2-S1-E,0.00,,ATOM,bine_2_s1_e.account_asset,20230110,2023-01-11 00:00:00.097,0.05,BINE
4,BINE-2-S1-E,"2,530.57",,AVAX,bine_2_s1_e.account_asset,20230110,2023-01-11 00:00:00.097,"32,217.30",BINE
5,BINE-2-S1-E,"93,931.83",,BAT,bine_2_s1_e.account_asset,20230110,2023-01-11 00:00:00.097,"18,471.85",BINE
6,BINE-2-S1-E,377.44,,BCH,bine_2_s1_e.account_asset,20230110,2023-01-11 00:00:00.097,"40,559.18",BINE
7,BINE-2-S1-E,40.81,,BNB,bine_2_s1_e.account_asset,20230110,2023-01-11 00:00:00.097,"11,316.13",BINE
8,BINE-2-S1-E,4.94,,BTC,bine_2_s1_e.account_asset,20230110,2023-01-11 00:00:00.097,"86,098.94",BINE
9,BINE-2-S1-E,267.14,,BUSD,bine_2_s1_e.account_asset,20230110,2023-01-11 00:00:00.097,267.21,BINE


## Compare the Ops EOD balance table with the balances reported in the account balance records